In [ ]:
import os

# Load environment variable
api_key = os.getenv("QWEN_API_KEY")

if api_key:
    print(f"API Key: found")
else:
    print("API key not found.")

if not os.getenv("DASHSCOPE_API_KEY"):
    os.environ["DASHSCOPE_API_KEY"] = api_key

In [ ]:
from langchain_qwq import ChatQwen
#qwen-flash"
llm = ChatQwen(
    model="qwen3-omni-flash",
    max_tokens=3_000,
    timeout=None,
    max_retries=2,
    # other params...
)

In [ ]:
messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French."
        "Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = llm.invoke(messages)
print(ai_msg.content)

In [ ]:
import openai
import os

# Load the OpenAI API key from environment variable
openai.api_key = os.getenv('OPENAI_API_KEY')

# Transcribe audio using Whisper API
def transcribe_audio_whisper(file_path):
    with open(file_path, "rb") as audio_file:
        response = openai.Audio.transcribe(
            model="whisper-1",  # You can specify the model like "whisper-1" for transcription
            file=audio_file
        )
    return response['text']

# Example: Transcribe an audio file
transcribed_text = transcribe_audio_whisper("test_voice.wav")
print("Transcribed Text: ", transcribed_text)
